In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="i", per_letter=True, sep=" ", start_row=None, end_row=None
)
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: 928898879916979767249474925511966989899657399958398618923999959898479982234896574
5856766795499834649 | Last line: 699479671999999899669382788445785359479768774769496811695989
3999957898943799975585881539179956768118

------

Convert to:

First line: [9, 2, 8, 8, 9, 8, 8, 7, 9, 9, 1, 6, 9, 7, 9, 7, 6, 7, 2, 4, 9, 4, 7, 4, 9, 2, 5,
5, 1, 1, 9, 6, 6, 9, 8, 9, 8, 9, 9, 6, 5, 7, 3, 9, 9, 9, 5, 8, 3, 9, 8, 6, 1, 8, 9, 2, 3, 9, 
9, 9, 9, 5, 9, 8, 9, 8, 4, 7, 9, 9, 8, 2, 2, 3, 4, 8, 9, 6, 5, 7, 4, 5, 8, 5, 6, 7, 6, 6, 7, 
9, 5, 4, 9, 9, 8, 3, 4, 6, 4, 9]

Last line: [6, 9, 9, 4, 7, 9, 6, 7, 1, 9, 9, 9, 9, 9, 9, 8, 9, 9, 6, 6, 9, 3, 8, 2, 7, 8, 8, 
4, 4, 5, 7, 8, 5, 3, 5, 9, 4, 7, 9, 7, 6, 8, 7, 7, 4, 7, 6, 9, 4, 9, 6, 8, 1, 1, 6, 9, 5, 9, 
8, 9, 3, 9, 9, 9, 9, 5, 7, 8, 9, 8, 9, 4, 3, 7, 9, 9, 9, 7, 5, 5, 8, 5, 8, 8, 1, 5, 3, 9, 1, 
7, 9, 9, 5, 6, 7, 6, 8, 1, 1, 8]

------

In [4]:
def new_mat(data):
    row_li = []
    for i in range(5):
        col_li = []
        for j in range(5):
            A = (np.array(data) + i + j) % 9
            A[A == 0] = 9
            col_li.append(A)
        row_li.append(np.hstack(col_li))
    new_data = np.vstack(row_li)
    return new_data


import networkx as nx
from networkx.algorithms.shortest_paths.weighted import single_source_dijkstra


def get_shortest_weighted_path(data):
    G = nx.DiGraph()  # or Graph, MultiGraph, MultiDiGraph, etc
    for i in range(len(data)):
        for j in range(len(data[0])):
            edge_li = []
            if i > 0:
                edge_li.append(((i, j), (i - 1, j), data[i - 1][j]))
            if j > 0:
                edge_li.append(((i, j), (i, j - 1), data[i][j - 1]))
            if i < len(data) - 1:
                edge_li.append(((i, j), (i + 1, j), data[i + 1][j]))
            if j < len(data[0]) - 1:
                edge_li.append(((i, j), (i, j + 1), data[i][j + 1]))

            G.add_weighted_edges_from(edge_li)
    return single_source_dijkstra(G, (0, 0), (len(data) - 1, len(data[0]) - 1))[0]


print(f"Answer to Q1: {get_shortest_weighted_path(data)}")
print(f"Answer to Q2: {get_shortest_weighted_path(new_mat(data))}")

Answer to Q1: 748

Answer to Q2: 3045

In [5]:
%timeit get_shortest_weighted_path(data)

301 ms ± 40 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit get_shortest_weighted_path(new_mat(data))

7.67 s ± 1.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def new_mat(data, scale=5):
    return np.vstack(
        [
            np.hstack(
                [
                    np.where(
                        (np.array(data) + i + j) % 9 == 0,
                        9,
                        (np.array(data) + i + j) % 9,
                    )
                    for j in range(scale)
                ]
            )
            for i in range(scale)
        ]
    )


import networkx as nx
from networkx.algorithms.shortest_paths.weighted import (
    dijkstra_path_length,
    single_source_dijkstra,
)


def get_shortest_weighted_path(data):
    G = nx.DiGraph()  # or Graph, MultiGraph, MultiDiGraph, etc
    data = np.pad(np.array(data), pad_width=1, constant_values=9999)
    for i in range(1, len(data) - 1):
        for j in range(1, len(data[0]) - 1):
            edge_li = []
            for target_node in [(i - 1, j), (i, j - 1), (i + 1, j), (i, j + 1)]:
                edge_li.append(
                    ((i, j), target_node, data[target_node[0]][target_node[1]])
                )
            G.add_weighted_edges_from(edge_li)
    return dijkstra_path_length(G, (1, 1), (len(data) - 2, len(data[0]) - 2))


print(f"Answer to Q1: {get_shortest_weighted_path(data)}")
print(f"Answer to Q2: {get_shortest_weighted_path(new_mat(data))}")

Answer to Q1: 748

Answer to Q2: 3045

In [8]:
%timeit get_shortest_weighted_path(data)

130 ms ± 3.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit get_shortest_weighted_path(new_mat(data))

4.05 s ± 237 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
